# XCEPTION MODEL 

In [1]:
import pandas as pd  # Import the pandas library for data manipulation
import numpy as np  # Import the NumPy library for numerical operations
import cv2  # Import OpenCV for image processing
import os  # Import the os module for operating system-related functions
from sklearn.model_selection import train_test_split  # Import train_test_split for splitting data
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder for encoding labels
from tensorflow import keras  # Import the TensorFlow Keras API for building deep learning models
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Import ImageDataGenerator for data augmentation
from tensorflow.keras.utils import to_categorical  # Import to_categorical for one-hot encoding
import random  # Import random module for generating random numbers
import tensorflow as tf  # Import TensorFlow
from tensorflow.keras.applications import Xception  # Import the Xception pre-trained model
from tensorflow.keras.models import Sequential  # Import the Sequential model
from tensorflow.keras.layers import Flatten, Dense, Dropout  # Import specific layers (Flatten, Dense, Dropout)
from tensorflow.keras.optimizers import Adam  # Import the Adam optimizer


In [2]:
# Load the CSV file containing image file names and class labels
df = pd.read_csv('image_label.csv')

In [3]:
# Define constants for image resizing and normalization
image_width = 128   # Width of the images after resizing to 128 pixels
image_height = 128  # Height of the images after resizing to 128 pixels
image_channels = 3  # Number of color channels in the images (RGB)


In [4]:
# Initialize empty lists to store preprocessed data
images = []   # List to store preprocessed images
labels = []   # List to store labels associated with the images

# Iterate through each row in the CSV file and load/encode images and labels
for index, row in df.iterrows():
    # Construct the path to the image file based on the 'Image_name' column in the DataFrame
    image_path = f"images/{row['Image_name']}.png"

    # Extract the label for the current image from the 'Plane' column in the DataFrame
    label = row['Plane']

    # Read the image using OpenCV
    image = cv2.imread(image_path)

    # Resize the image to the specified dimensions (image_width x image_height)
    image = cv2.resize(image, (image_width, image_height))

    # Normalize the pixel values of the image to a range between 0 and 1 by dividing by 255.0
    image = image / 255.0

    # Append the preprocessed image and its corresponding label to the respective lists
    images.append(image)
    labels.append(label)


In [5]:
# Encode class labels into numerical values
label_encoder = LabelEncoder()  # Create a LabelEncoder object to convert labels to numerical values
labels_encoded = label_encoder.fit_transform(labels)  # Encode the labels

label_map = {}  # Create an empty dictionary to map numerical labels back to their original values

# Iterate through the encoded labels and their original values
for i, x in enumerate(labels_encoded):
    if x not in label_map:
        label_map[x] = labels[i]  # Map the numerical label 'x' to its original label 'labels[i]'


In [6]:
# Initialize an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,       # Rotate the image by up to 10 degrees
    shear_range=0.2,         # Apply shear transformation with a maximum shear angle of 0.2 radians
    zoom_range=0.2,          # Zoom in/out on the image by up to 20%
    horizontal_flip=True,    # Flip the image horizontally (left to right) with a 50% chance
    fill_mode='nearest'      # Fill in newly created pixels after rotation or shearing using the nearest pixel's value
)


In [7]:
# Initialize lists to store augmented data
augmented_images = []  # List to store augmented images
augmented_labels = []  # List to store labels for augmented images

# Iterate through the original images and their corresponding labels
for i, (image, label) in enumerate(zip(images, labels_encoded)):
    image = np.expand_dims(image, axis=0)  # Add an extra dimension to the image (batch dimension)
    label = np.array([label])  # Convert the label to a NumPy array

    amount = 0  # Initialize a counter to keep track of the number of augmented images
    for batch in datagen.flow(image, label, batch_size=1):
        augmented_image, augmented_label = batch  # Extract augmented image and label
        augmented_image = augmented_image[0]  # Remove the extra batch dimension
        augmented_label = augmented_label[0]  # Convert label back to a single value

        augmented_images.append(augmented_image)  # Append augmented image to the list
        augmented_labels.append(augmented_label)  # Append augmented label to the list

        amount += 1  # Increment the counter
        if amount >= random.choice([5, 6, 7, 8, 9, 10]):  # Randomly decide to stop augmentation
            break

# Extend the original lists with augmented data
images.extend(augmented_images)  # Add augmented images to the original list of images
labels_encoded = np.concatenate([labels_encoded, augmented_labels])  # Concatenate augmented labels to the original labels


In [8]:
# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels_encoded, test_size=0.2, random_state=42, stratify=labels_encoded)

# Further split the temporary data into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Convert data and labels to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

# Print the shapes of the data and labels
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)
print("Testing data shape:", X_test.shape)
print("Training label shape:", y_train.shape)
print("Validation label shape:", y_val.shape)
print("Testing label shape:", y_test.shape)

# Define the number of classes
num_classes = 4

# Convert labels to one-hot encoded format
y_train_onehot = to_categorical(y_train, num_classes)
y_val_onehot = to_categorical(y_val, num_classes)
y_test_onehot = to_categorical(y_test, num_classes)

# Print the shapes of the one-hot encoded labels
print("Training label shape (one-hot encoded):", y_train_onehot.shape)
print("Validation label shape (one-hot encoded):", y_val_onehot.shape)
print("Testing label shape (one-hot encoded):", y_test_onehot.shape)


Training data shape: (10188, 128, 128, 3)
Validation data shape: (1273, 128, 128, 3)
Testing data shape: (1274, 128, 128, 3)
Training label shape: (10188,)
Validation label shape: (1273,)
Testing label shape: (1274,)
Training label shape (one-hot encoded): (10188, 4)
Validation label shape (one-hot encoded): (1273, 4)
Testing label shape (one-hot encoded): (1274, 4)


# Model Training

In [9]:
# Define a function to build the deep learning model
def build_model(input_shape, num_classes):
    model = Sequential()
    model.add(Xception(weights='imagenet', include_top=False, input_shape=input_shape))  # Use Xception as a base model
    model.add(Flatten())  # Flatten the output from the convolutional base
    model.add(Dense(256, activation='relu'))  # Add a fully connected layer with 256 units and ReLU activation
    model.add(Dropout(0.5))  # Add dropout to reduce overfitting
    model.add(Dense(128, activation='relu'))  # Add another fully connected layer with 128 units and ReLU activation
    model.add(Dense(num_classes, activation='softmax'))  # Output layer with softmax activation for classification
    return model

# Define the input shape and number of classes
image_shape = (image_width, image_height, 3)  # Assuming these dimensions match your image data
num_classes = num_classes  # Previously defined as 4

# Build the model
model = build_model(image_shape, num_classes)

# Compile the model with an optimizer, loss function, and evaluation metric
model.compile(optimizer=Adam(learning_rate=0.0001),  # Adam optimizer with a specified learning rate
              loss='categorical_crossentropy',  # Categorical cross-entropy loss for multiclass classification
              metrics=['accuracy'])  # Monitor accuracy during training

# Define early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,  # Stop training if validation loss does not improve for 5 consecutive epochs
    restore_best_weights=True  # Restore the best model weights when training stops
)

# Train the model on the training data with validation data for 10 epochs
history = model.fit(X_train, y_train_onehot,  # Training data and labels
                    epochs=10,  # Number of training epochs
                    validation_data=(X_val, y_val_onehot),  # Validation data and labels
                    callbacks=[early_stopping])  # Use early stopping callback

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test_onehot)  # Evaluate on test data
print(f'Test accuracy: {test_acc}')  # Print test accuracy


83683744/83683744 [==============================] - 19s 0us/step
Epoch 1/10
319/319 [==============================] - 1320s 4s/step - loss: 0.4016 - accuracy: 0.8611 - val_loss: 0.2468 - val_accuracy: 0.9285
Epoch 2/10
319/319 [==============================] - 1297s 4s/step - loss: 0.0737 - accuracy: 0.9765 - val_loss: 0.0932 - val_accuracy: 0.9733
Epoch 3/10
319/319 [==============================] - 1314s 4s/step - loss: 0.0352 - accuracy: 0.9891 - val_loss: 0.0594 - val_accuracy: 0.9811
Epoch 4/10
319/319 [==============================] - 1320s 4s/step - loss: 0.0222 - accuracy: 0.9942 - val_loss: 0.0637 - val_accuracy: 0.9866
Epoch 5/10
319/319 [==============================] - 1314s 4s/step - loss: 0.0191 - accuracy: 0.9951 - val_loss: 0.0513 - val_accuracy: 0.9890
Epoch 6/10
319/319 [==============================] - 1298s 4s/step - loss: 0.0302 - accuracy: 0.9921 - val_loss: 0.1761 - val_accuracy: 0.9670
Epoch 7/10
319/319 [==============================] - 1396s 4s/step - 

In [10]:
model.save('model.h5')#HDF5 format. HDF5 stands for Hierarchical Data Format version 5

In [11]:
label_map = {
    0: 'Fetal brain',
    1: 'Fetal femur',
    2: 'Fetal thorax',
    3: 'Fetal abdomen'
}

Fetal brain: Refers to the developing brain of an unborn baby during pregnancy.

Fetal femur: The thigh bone of a developing fetus.

Fetal thorax: The chest area of a developing fetus, including the ribcage and associated structures.

Fetal abdomen: The abdominal region of a developing fetus, which includes the stomach and other organs in the belly area.

In [12]:
model = keras.models.load_model('model.h5')

In [13]:
images = []  # List to store preprocessed images
classes = []  # List to store predicted class names

dir_path = 'images'  # Path to the directory containing the images

# Loop through each file in the specified directory
for file_name in os.listdir(dir_path):
    image_path = os.path.join(dir_path, file_name)  # Create the full path to the image file
    image = cv2.imread(image_path)  # Read the image using OpenCV
    image = cv2.resize(image, (image_width, image_height))  # Resize the image
    image = image / 255.0  # Normalize pixel values to the range [0, 1]
    images.append(image)  # Append the preprocessed image to the 'images' list

    # Use the 'model' to make predictions on the image
    predictions = model.predict(np.expand_dims(image, axis=0))  # Expand dimensions to match model input shape
    predicted_class = np.argmax(predictions)  # Get the index of the predicted class
    predicted_class_name = label_map.get(predicted_class, 'Unknown')  # Map index to class name using 'label_map'
    classes.append(predicted_class_name)  # Append the predicted class name to the 'classes' list

    # Print the predicted class name for this image
    print(f'Predicted class: {predicted_class_name}')


1/1 [==============================] - 2s 2s/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 117ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 130ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 107ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 105ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 110ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 142ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 114ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 114ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 145ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 133ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 134ms/step
Predicted class: Fetal brain
1/1 [=====

1/1 [==============================] - 0s 145ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 134ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 132ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 143ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 145ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 119ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 161ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 153ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 157ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 161ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 147ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 130ms/step
Predicted class: Fetal thorax
1/1 

1/1 [==============================] - 0s 130ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 144ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 126ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 121ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 138ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 133ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 149ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 181ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 227ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 216ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 1s 632ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 419ms/step
Predicted class: Fetal femur
1/1 

1/1 [==============================] - 0s 105ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 114ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 108ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 120ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 127ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 122ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 121ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 115ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 127ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 135ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 122ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 117ms/step
Predicted class: Fetal femur
1/1 [=====

1/1 [==============================] - 0s 130ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 128ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 141ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 159ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 141ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 123ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 131ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 116ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 125ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 125ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 121ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 116ms/step
Predicted class: Fetal femur
1/1 [==

1/1 [==============================] - 0s 116ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 116ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 109ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 116ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 124ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 113ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 129ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 120ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 121ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 124ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 120ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 154ms/step
Predicted class: Fetal brain
1/1 [======

1/1 [==============================] - 0s 127ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 124ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 122ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 123ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 135ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 133ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 150ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 167ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 135ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 132ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 136ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 136ms/step
Predicted class: Fetal thorax
1/1 [==

1/1 [==============================] - 0s 114ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 121ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 123ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 127ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 130ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 133ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 113ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 121ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 126ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 118ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 125ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 119ms/step
Predicted class: Fetal femur
1/1

1/1 [==============================] - 0s 207ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 192ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 196ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 168ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 214ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 213ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 151ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 142ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 139ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 123ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 134ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 133ms/step
Predicted class: Fetal brain
1/1 [=======

1/1 [==============================] - 0s 70ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 72ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 73ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 67ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 69ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 69ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 68ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 67ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 71ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 69ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 67ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 70ms/step
Predicted class: Fetal thorax
1/1 [======================

In [14]:
# Define parameters for the collage layout
num_rows = 10  # Number of rows in the collage
num_cols = 8   # Number of columns in the collage
image_spacing = 60  # Spacing between images in pixels
image_height, image_width, _ = images[0].shape  # Get the dimensions of the individual images
collage_width = (image_width + image_spacing) * num_cols  # Calculate the width of the collage
collage_height = (image_height + image_spacing) * num_rows  # Calculate the height of the collage
collage = np.zeros((collage_height, collage_width, 3), dtype=np.uint8)  # Create an empty collage as a NumPy array

# Define font and text parameters for labeling
font = cv2.FONT_HERSHEY_SIMPLEX  # Font type
font_scale = 0.6  # Font scale factor
font_thickness = 2  # Font thickness
font_color = (255, 255, 255)  # Font color (white)

# Iterate through rows and columns to populate the collage
for i in range(num_rows):
    for j in range(num_cols):
        index = i * num_cols + j  # Calculate the index of the current image
        if index < len(images):  # Check if there are more images to add
            sample_image = (images[index] * 255.0).astype(np.uint8)  # Convert the image back to uint8 format
            y_offset = i * (image_height + image_spacing)  # Calculate the y-offset for this image
            x_offset = j * (image_width + image_spacing)   # Calculate the x-offset for this image
            # Copy the sample image into the collage at the specified location
            collage[y_offset:y_offset + image_height, x_offset:x_offset + image_width] = sample_image
            class_label = classes[index]  # Get the class label for this image
            # Calculate the position to display the class label centered below the image
            text_size, _ = cv2.getTextSize(class_label, font, font_scale, font_thickness)
            text_x = x_offset + (sample_image.shape[1] - text_size[0]) // 2
            text_y = y_offset + sample_image.shape[0] + text_size[1] + 5
            # Add the class label text to the collage image
            cv2.putText(collage, class_label, (text_x, text_y), font, font_scale, font_color, font_thickness)


In [15]:
cv2.imwrite('Final_Output_Xception.jpg', collage)#using OpenCV's imwrite function

True